In [1]:
import sympy as sm

# Our Model

Our model is the classic solowmodel, but instead of having a fixed savings rate, in each period the savings rate is decided by a micro-household optimization problem 

**Micro-foundation of the savingsrate** 

Representative household utilityfunction in single periode:
$$
u\left(c_{t}\right)=\frac{c^{1-\theta}-1}{1-\theta}
$$
When the household sums its total utility for mulitple future periods, it disocunts future consumption for immediate consumption by $1-\beta$ so utility for all periods from period 0 to period T, given a vector C of consumption in each period is:
$$
U\left(C\right)=\sum_{t=0}^{T}\beta^{t}\frac{c_{t}^{1-\theta}-1}{1-\theta}
$$
We assume that the household understands how the solow model works, e.g. how production for consumption is made and how capital accumulation works.
$$
Y_{t}=AK_{t}^{\alpha}L_{t}^{1-\alpha} 
\Rightarrow c_{t}\cdot L_{t} = \left(1-s_{K_t}\right)\cdot AK_{t}^{\alpha}L_{t}^{1-\alpha} 
$$ $$
K_{t+1}=s_{Kt}\cdot Y_{t}+\left(1+\delta\right)K_{t}
$$
$$
L_{t+1}=\left(1+n\right)L_{t}\
$$
Rewriting to per capita terms we get:
$$
y_{t}=A k_{t}^{\alpha} \Rightarrow
c_{t} = \left(1-s_{Kt}\right) A k_{t}^{\alpha}
$$ $$
k_{t+1}= \frac{1}{1+n} \left(s_{Kt}\cdot A k_{t}^{\alpha} + \left(1+\delta\right)k_{t} \right)
$$
This is out 'budget constraint' more precisely defined as production constraints for all periods 



So the final optimizationproblem is:
$$
\max_{s_{K_t}}U\left(C\right)=\sum_{t=0}^{T}\beta^{t}\frac{c_{t}^{1-\theta}-1}{1-\theta}
$$
$$ s.t.$$$$
c_{t} = \left(1-s_{Kt}\right) A k_{t}^{\alpha},\;t\in [0,T]
$$$$
k_{t}=
\begin{cases}
k_{0}& \text{if }t=0\\
\frac{1}{1+n} \left(s_{Kt-1}\cdot A k_{t-1}^{\alpha} + \left(1+\delta\right)k_{t-1} \right)& \text{if }t\in\left[1,T\right]
\end{cases}
$$$$
k_{0} \text{ given}
$$

**Macro foundations**

We use the basic solow model but with an adjustable savingsrate for each period. Rewritten we got the following capital-accumulation pr. capita:

$$
k_{t+1}= \frac{1}{1+n} \left(s_{Kt}\cdot A k_{t}^{\alpha} + \left(1+\delta\right)k_{t} \right)
$$$$
y_{t}=A k_{t}^{\alpha}
$$
For a fixed savings rate we know that this model has convergenge to steady state. With the adjustable micro-foundated savingsrate we do not yet know if we have convergens, but we do not that convergence will imply, $k_{t+1}=k_{t}=k^{*}$ thus we can we use sympy to derive:
$$
k^{*}= \frac{1}{1+n} \left(s_{Kt}\cdot A {k^{*}}^{\alpha} + \left(1+\delta\right)k^{*} \right)
$$
